<hr style="border:none;height:6px;background:#fff;margin:1em 0;">


<div style="text-align: center;">
  <h1>Empirical Asset Pricing with ML in Europe</h1>
  <h3>HEC Liege</h3>
  <h4><em>Lucas Dubois and Myriam Lamborelle</em></h4>
</div>

<hr style="border:none;height:6px;background:#fff;margin:1em 0;">


In [988]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import matplotlib as mpl
from statsmodels.stats import diagnostic
from statsmodels.stats.sandwich_covariance import cov_hac
from statsmodels.stats.sandwich_covariance import se_cov
from stargazer.stargazer import Stargazer
from linearmodels.panel import PanelOLS
from linearmodels.panel import PooledOLS
import statsmodels.formula.api as smf
import warnings
import re

In [990]:
path="/Users/lucasdubois/Desktop/LaTeX/EAP-ML/CODE/DATA/"

In [992]:
figure_path="/Users/lucasdubois/Desktop/LaTeX/EAP-ML/CODE/Images/"

In [994]:
offwhite = (230/255, 230/255, 220/255)
midnight = (0/255, 22/255, 36/255)
steelblue = (171/255, 193/255, 223/255)
primaryred = (127/255, 20/255, 22/255)
harmonizedblue =(48/255,88/255,140/255)

<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>1</small>&nbsp;&nbsp;&nbsp;&nbsp;Database:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">








<div style="text-align: align;">
  <h3> <small>1.1</small>&nbsp;&nbsp;&nbsp;&nbsp;Stocks:</h3>
</div>

In [1000]:
df = pd.read_excel(path + "stoxx600.xlsx")

df.rename(columns={df.columns[0]: "Date"}, inplace=True)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

In [1001]:
df = df.loc[:, ~df.columns.str.startswith("#ERROR")]

In [1002]:
df_long = df.melt(
    id_vars="Date",
    var_name="col",
    value_name="value"
)

In [1003]:
df_long[["Stock", "Datatype"]] = (
    df_long["col"]
    .str.split(" - ", n=1, expand=True)
)

In [1004]:
df_panel = (
    df_long
    .pivot_table(
        index=["Date", "Stock"],
        columns="Datatype",
        values="value"
    )
    .reset_index()
)


In [1005]:
df_panel = df_panel.sort_values(["Stock", "Date"])

print(df_panel.head())
print(df_panel.tail())
print(df_panel.info())


Datatype       Date     Stock  DIVIDEND YIELD  MARKET VALUE  \
0        2005-01-01  3I GROUP            2.13       4085.77   
435      2005-02-01  3I GROUP            2.00       4355.70   
871      2005-03-01  3I GROUP            2.04       4279.01   
1307     2005-04-01  3I GROUP            2.11       4134.96   
1743     2005-05-01  3I GROUP            2.23       3913.99   

Datatype  MRKT VALUE TO BOOK   PER  PRICE INDEX  TOT RETURN IND  \
0                       1.12  30.4        244.9          302.56   
435                     1.20  32.4        261.0          322.55   
871                     1.18  31.8        256.4          316.87   
1307                    1.03  30.7        247.4          305.74   
1743                    0.98  29.1        234.6          289.84   

Datatype  TURNOVER BY VALUE  TURNOVER BY VOLUME  
0                  340749.3             83500.9  
435                338096.4             81321.8  
871                446679.1            100759.9  
1307              

In [1006]:
df_panel["Stock"].nunique()

599

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>1.2</small>&nbsp;&nbsp;&nbsp;&nbsp;Risk-Free:</h3>
</div>

In [1009]:
rf_df = pd.read_excel(path + "EURIBOR.xlsx", header=None)

In [1010]:
rf_df.columns = rf_df.iloc[0]
rf_df = rf_df.iloc[1:].reset_index(drop=True)


In [1011]:
rf_df.head()
rf_df.columns


Index(['DATE', 'TIME PERIOD', 'OBS.VALUE'], dtype='object', name=0)

In [1012]:
rf_df = rf_df.rename(columns={
    "DATE": "Date",
    "OBS.VALUE": "Rf"
})
rf_df = rf_df.drop(columns=["TIME PERIOD"])

In [1013]:
rf_df["Date"] = pd.to_datetime(rf_df["Date"])
rf_df["Rf"] = rf_df["Rf"].astype(float) / 100 / 12

In [1014]:
rf_df["Date"] = rf_df["Date"] + pd.offsets.MonthBegin(1)


In [1015]:
rf_df.head

<bound method NDFrame.head of 0         Date        Rf
0   2005-01-01  0.001806
1   2005-02-01  0.001759
2   2005-03-01  0.001753
3   2005-04-01  0.001753
4   2005-05-01  0.001754
..         ...       ...
247 2025-08-01  0.001577
248 2025-09-01  0.001575
249 2025-10-01  0.001581
250 2025-11-01  0.001589
251 2025-12-01  0.001588

[252 rows x 2 columns]>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>1.3</small>&nbsp;&nbsp;&nbsp;&nbsp;Merging:</h3>
</div>

In [1018]:
df_almost = df_panel.merge(
    rf_df,
    on="Date",
    how="left"
)


In [1019]:
df_almost["Rf"].describe()

count    133070.000000
mean          0.000881
std           0.001363
min          -0.000497
25%          -0.000308
50%           0.000216
75%           0.001866
max           0.004026
Name: Rf, dtype: float64

In [1020]:
df_almost.to_csv(path + "Data1.csv", index=False)

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>1.4</small>&nbsp;&nbsp;&nbsp;&nbsp;Data Management</h3>
</div>

In [1023]:
cols = df_almost.columns.tolist()
cols_no_date = [c for c in cols if c != "Date"]


In [1024]:
df_long = df_almost.melt(
    id_vars=["Date", "Stock"],
    var_name="Variable",
    value_name="value"
)

In [1028]:
df_long.columns


Index(['Date', 'Stock', 'Variable', 'value'], dtype='object')

In [1029]:
df_long.columns
df_long.head()

,Date,Stock,Variable,value
0,2005-01-01,3I GROUP,DIVIDEND YIELD,2.13
1,2005-02-01,3I GROUP,DIVIDEND YIELD,2.00
2,2005-03-01,3I GROUP,DIVIDEND YIELD,2.04
3,2005-04-01,3I GROUP,DIVIDEND YIELD,2.11
4,2005-05-01,3I GROUP,DIVIDEND YIELD,2.23


In [1030]:
df_panel = (
    df_long
    .pivot_table(
        index=["Date", "Stock"],
        columns="Variable",
        values="value"
    )
    .reset_index()
)

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>1.5</small>&nbsp;&nbsp;&nbsp;&nbsp;Excess Returns:</h3>
</div>

In [1034]:
df_panel["RET"] = (
    df_panel
    .groupby("Stock")["TOT RETURN IND"]
    .pct_change(fill_method=None)
)

In [1047]:
df_panel["RET_FWD"] = df_panel.groupby("Stock")["RET"].shift(-1)
df_panel["EXCESS_RET_FWD"] = df_panel["RET_FWD"] - df_panel["Rf"]


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>1.5</small>&nbsp;&nbsp;&nbsp;&nbsp;Firm Characteristics:</h3>
</div>

In [1074]:
def safe_log(x):
    out = np.full_like(x, np.nan, dtype="float64")
    mask = x > 0
    out[mask] = np.log(x[mask])
    return out

In [1076]:
df_panel["SIZE"] = safe_log(df_panel["MARKET VALUE"])
df_panel["BM"] = 1 / df_panel["MRKT VALUE TO BOOK"]
df_panel["DY"] = df_panel["DIVIDEND YIELD"] / 100
df_panel["EY"] = 1 / df_panel["PER"]
df_panel["MOM3"] = (
    df_panel
    .groupby("Stock")["PRICE INDEX"]
    .pct_change(2, fill_method=None)
)

df_panel["MOM12"] = (
    df_panel
    .groupby("Stock")["PRICE INDEX"]
    .pct_change(11, fill_method=None)
)

df_panel["VOL12"] = (
    df_panel
    .groupby("Stock")["RET"]
    .rolling(12)
    .std()
    .reset_index(level=0, drop=True)
)
df_panel["TURN_VAL"] = safe_log(df_panel["TURNOVER BY VALUE"])
df_panel["TURN_VOL"] = safe_log(df_panel["TURNOVER BY VOLUME"])
df_panel["LOG_PRICE"] = safe_log(df_panel["PRICE INDEX"])


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>1.5</small>&nbsp;&nbsp;&nbsp;&nbsp;Firm Characteristics:</h3>
</div>

In [1078]:
df_panel["SIZE_BM"]   = df_panel["SIZE"] * df_panel["BM"]
df_panel["SIZE_MOM"]  = df_panel["SIZE"] * df_panel["MOM12"]
df_panel["SIZE_VOL"]  = df_panel["SIZE"] * df_panel["VOL12"]
df_panel["SIZE_LIQ"]  = df_panel["SIZE"] * df_panel["TURN_VAL"]
df_panel["BM_MOM"] = df_panel["BM"] * df_panel["MOM12"]
df_panel["BM_VOL"] = df_panel["BM"] * df_panel["VOL12"]
df_panel["BM_LIQ"] = df_panel["BM"] * df_panel["TURN_VAL"]
df_panel["MOM_VOL"] = df_panel["MOM12"] * df_panel["VOL12"]
df_panel["MOM_LIQ"] = df_panel["MOM12"] * df_panel["TURN_VAL"]
df_panel["MOM_PRICE"] = df_panel["MOM12"] * df_panel["LOG_PRICE"]
df_panel["EY_MOM"] = df_panel["EY"] * df_panel["MOM12"]
df_panel["DY_SIZE"] = df_panel["DY"] * df_panel["SIZE"]

In [1084]:
df_FINAL= df_panel

In [1088]:
df_FINAL.to_csv(path + "FINAL.csv", index=False)